In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
print("at directory:", os.getcwd())
print("changing to root directory")
os.chdir("../../..")
print("at directory:", os.getcwd())

at directory: /Users/wilka/git/research/jaxneurorl/projects/humansf/analysis
changing to root directory
at directory: /Users/wilka/git/research/jaxneurorl


In [3]:
# !pip install google-cloud-storage
# !pip install git+https://github.com/wcarvalho/nicewebrl --no-deps
# !pip install ipywidgets
# !pip install polars
# !pip install nicegui

In [4]:
import jax
import jax.numpy as jnp
import functools
from functools import partial
import pandas as pd
import numpy as np
from pprint import pprint
import glob
from glob import glob
import matplotlib.pyplot as plt
import polars as pl
from tqdm.notebook import tqdm

from projects.humansf import data_loading
from projects.humansf import housemaze_analysis
from projects.humansf import data_loading
from projects.humansf import housemaze_trainer
from projects.humansf import housemaze_experiments
from projects.humansf import networks
from housemaze.human_dyna import mazes

from housemaze import renderer
from housemaze import utils
from housemaze.human_dyna import multitask_env as maze
from jaxneurorl import launcher
from jaxneurorl.agents import value_based_basics as vbb


from projects.humansf import download_data

wandb: WARNING `wandb.require('core')` is redundant as it is now the default behavior.


No file specified for image dict.
Using: /Users/wilka/git/research/jaxneurorl/libraries/housemaze/housemaze/image_data.pkl
No file specified for image dict.
Using: /Users/wilka/git/research/jaxneurorl/libraries/housemaze/housemaze/image_data.pkl
No file specified for groups.
Using: /Users/wilka/git/research/jaxneurorl/libraries/housemaze/housemaze/human_dyna/list_of_groups.npy


/Users/wilka/miniforge3/envs/jaxneurorl/lib/python3.10/site-packages/distrax/_src/utils/conversion.py:145: UserWarning: Explicitly requested dtype <class 'jax.numpy.float64'> requested in astype is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  return x.astype(jnp.float_)


In [8]:
# !ls -lhtr ~/git/research/results/human_dyna/user_data/*

In [19]:
# !rm ~/git/research/results/human_dyna/user_data/*3714894224_name* ~/git/research/results/human_dyna/user_data/*2631242498_name*

In [9]:
from projects.humansf.download_data import download_user_files

bucket_name = "human-dyna"
prefix = "data/"
#pattern = "data/data_user=*_name=r0-v2*debug=0.json"
human_data_pattern = "data/data_user=*r0-exp2-obj1-v1*debug=0.json"
destination_folder = "/Users/wilka/git/research/results/human_dyna/user_data/exp2"

download_user_files(bucket_name, prefix, human_data_pattern, destination_folder)

Downloaded: data/data_user=1067476058_name=r0-exp2-obj1-v1_exp=2_debug=0.json to /Users/wilka/git/research/results/human_dyna/user_data/exp2/data_user=1067476058_name=r0-exp2-obj1-v1_exp=2_debug=0.json
Downloaded: data/data_user=1098132550_name=r0-exp2-obj1-v1_exp=2_debug=0.json to /Users/wilka/git/research/results/human_dyna/user_data/exp2/data_user=1098132550_name=r0-exp2-obj1-v1_exp=2_debug=0.json
Downloaded: data/data_user=1326608468_name=r0-exp2-obj1-v1_exp=2_debug=0.json to /Users/wilka/git/research/results/human_dyna/user_data/exp2/data_user=1326608468_name=r0-exp2-obj1-v1_exp=2_debug=0.json
Downloaded: data/data_user=1415357222_name=r0-exp2-obj1-v1_exp=2_debug=0.json to /Users/wilka/git/research/results/human_dyna/user_data/exp2/data_user=1415357222_name=r0-exp2-obj1-v1_exp=2_debug=0.json
Downloaded: data/data_user=1525319157_name=r0-exp2-obj1-v1_exp=2_debug=0.json to /Users/wilka/git/research/results/human_dyna/user_data/exp2/data_user=1525319157_name=r0-exp2-obj1-v1_exp=2_deb

In [14]:
data_dir = '/Users/wilka/git/research/results/human_dyna/'
files = f'{data_dir}/user_data/exp2/*obj1*v1*.json'
user_base_path = f'{data_dir}/user_data'
files = glob(files)
len(files)

32

In [18]:
import json
nfinished = 0
for idx, file in enumerate(files[4:]):
    with open(file, 'r') as f:
        try:
            data = json.load(f)
        except Exception as e:
            print('-'*25)
            print(idx, os.path.basename(file))
            print(e)
            continue
        finished = data[-1].get("finished", False)
        if finished:
            nfinished += 1
            print('-'*25)
            print(idx, os.path.basename(file))
            print("\n" + data[-1]['feedback'] or "N/A")
            print('bonus:', data[-1]['bonus'])
        # else:
        #     print("NOT FINISHED")

-------------------------
2 data_user=2672838701_name=r0-exp2-obj1-v1_exp=2_debug=0.json


bonus: 3
-------------------------
4 data_user=2958212813_name=r0-exp2-obj1-v1_exp=2_debug=0.json

Thank you,
bonus: 3
-------------------------
5 data_user=444968707_name=r0-exp2-obj1-v1_exp=2_debug=0.json

The movement of the arrow was unresponsive and caused me to make mistakes.
bonus: 3
-------------------------
6 data_user=606867615_name=r0-exp2-obj1-v1_exp=2_debug=0.json

it was so laggy and hard to control
bonus: 3
-------------------------
7 data_user=3786090989_name=r0-exp2-obj1-v1_exp=2_debug=0.json


bonus: 3
-------------------------
9 data_user=1921995450_name=r0-exp2-obj1-v1_exp=2_debug=0.json

God this was terrible, to the point of being exploitative. What's wrong with this lab??  Anyway, the "experiment" screwed up the first time, randomly, requiring restarting. Then, the controls were crappy; didn't register arrow presses, and then the 'runner' would jump to a different position 

In [59]:
user_df, user_data = data_loading.make_all_episode_data(
    files=files,
    example_timestep=example_timestep,
    base_path=user_base_path,
    overwrite=True,
)

Processing files:   0%|          | 0/2 [00:00<?, ?it/s]

Filtered 46 data point


Processing episodes:   0%|          | 0/51 [00:00<?, ?it/s]

Filtered 65 data point


Processing episodes:   0%|          | 0/49 [00:00<?, ?it/s]